<div style="text-align:right;color:blue">version id: __VERSION_ID__</div>

# The JupyterLab Debugger

In the previous notebook we have tried to gain insights into the behaviour of the code by adding print statements. This often works well, but there is a more elegant method: we can use a dedicated debugger that allows us to step through the code line-by-line while inspecting the values of variables. Although Python comes with a [built-in debugger](https://docs.python.org/3/library/pdb.html), this is quite hard to use. for this reason we will look at the JupyterLab debugger, which provides a more intuitive graphical interface. In this notebook we will discuss the JupyterLab debugger interface and look at how it can be used to find a subtle bug in an example code.

If you prefer to watch a live demonstration of the material covered in this notebook, please have a look at the [this video on Panopto](https://uniofbath.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=b28ecd3f-9b2d-4162-96fa-ae2400c24948).

<hr style="height: 2px">

### What you will learn
In this notebook we will cover the following topics:

* Example code: partial summation
* Invoking the JupyterLab Debugger
* Setting breakpoints
* Stepping through code

<hr style="height: 2px">

*&#169; Eike Mueller, University of Bath 2019-2021. These lecture notes are copyright of Eike Mueller, University of Bath. They are provided exclusively for educational purposes at the University and are to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

## The JupyterLab debugger
To illustrate the working of the JupyterLab debugger, we will look at the following problem:

Imagine that we have a numpy array $a$ of $n$ numbers $a_0,a_1,\dots,a_{n-1}$. We want to construct another numpy array $b$ of the same length, such that the element $b_j$ contains the *partial sum* of the squares of all elements in $a$ up to $j$. More specifically, we have that

$$
b_j = \sum_{k=0}^j a_k^2 \qquad\text{for $j=0,1,2,\dots,n-1$}
$$

For example, if $a=[2,1,3]$, then $b=[2^2,2^2+1^2,2^2+1^2+3^2]=[4,5,14]$. Here is some pseudocode for constructing $b$ from $a$:

### Function to compute partial sum of array
**Input:** Array $a$, **Output** Array $b$ with $b_j=\sum_{k=0}^{j}a_k^2$
1. Set $n\mapsto \text{length of $a$}$
1. Set $b\mapsto a$ (copy array $a$ to initialise $b$ of same length)
1. **for $j=0,1,2,\dots,n-1$ do**
1. $~~~~$ Set $b_j\mapsto 0$
1. $~~~~$ **for $k=0,1,\dots,j$ do**
1. $~~~~~~~~$ Update $b_j\mapsto b_j + a_k^2$
1. $~~~~$ **end for**
1. **end for**


This is implemented in the following Python code. To illustrate some features of the debugger, an additional function `square()` for computing the square of a given number has been introduced.

In [9]:
import numpy as np

def square(x):
    '''Square a number
    
    input: number x
    output: the square x^2
    '''
    g = x**2
    return g

def partial_sum(a):
    '''Compute the partial sum of an array a
    
    input: array a of length n
    output: array b of length n, such that b_j = \sum_{k=0}^{j} a_k^2
    '''
    n = len(a)
    b = np.asarray(a)
    for j in range(n):
        b[j] = 0
        for k in range(j+1):            
            b[j] += square(a[k])
    return b

Running the code, we see that it clearly does not produce correct results. For example:

In [ ]:
b = partial_sum(np.array([2,1,3]))
print (b)

Let's try to find and fix the bug in this code with the JupyterLab debugger.

### Enabling the JupyterLab debugger
To enable the debugger, we first have to make sure that the Python 3 (ipykernel) kernel is selected. This should be the default setting, but if the kernel is not chosen, click on the kernel selector in the upper right corner of the notebook as shown in the following figure:

![Switching kernel](./figures/switch_kernel.png)

This will open a dialog with a dropdown menu, from which the Python 3 (ipykernel) kernel can be chosen:

![Selecting Python 3 (ipykernel) kernel](./figures/select_kernel_1.png)
![Selecting Python 3 (ipykernel) kernel](./figures/select_kernel_2.png)

The name of the kernel in the upper right corner should now have changed to "Python 3 (ipykernel)". To enable the debugger click on the little bug symbol, which should change colour from gray to orange as shown here:

![Enabling the debugger](./figures/bug_symbol.png)

Note that line numbers have appeared on the left of each code cell as in the following example:

![Numbered code lines](./figures/numbered_code.png)

### Setting breakpoints
Clicking on a particular line number will set a **breakpoint**. As shown in the following figure, the line is now marked with a red dot, and an entry has appeared in the "BREAKPOINTS" list on the right hand side of the window:

![Setting a breakpoint](./figures/setting_breakpoint.png)

When running code, Python will stop execution at breakpoints and enter the debugger, as we will see in the following example.

Go back to the cell above that defines the `partial_sum()` function above, and set a breakpoint in line 21. Once you have done this, run the cell above that calls this function and contains the code

```Python
b = partial_sum(np.array([2,1,3]))
print (b)
```

The screen should now change to something as in the following figure:

![Active debugger](./figures/active_debugger.png)

The background colour of the line at which we set the breakpoint has changed to gray, and Python has stopped execution of the code at this point (note that the cell that calls the `partial_sum()` function is still marked as running as indicated by the `[*]:` symbol on the left).

On the right side we see three additional panes, marked in green, orange and blue (if you do not see these panes, select the 'bug' symbol just below the 'gears' symbol):

* The **VARIABLES** pane (highlighted in green) lists the current values of all variables. Make sure that you click on the three horizontal lines in the upper right corner of the VARIABLES pane to display the variables in 'Table View'. As expected, there are four variables `a` (the original array), `b` (the new array) `j` (the loop index) and `n` (the length of `a`). Since we stopped in the first iteration of the loop, `j` is equal to 0 and `a` is identical to `b`. This is what we would expected, since we copied `b` from `a` with the `b=np.asarray(a)` statement.

* The **CALLSTACK** pane (highlighted in orange) tells us which function Python is currently looking at. Not surprisingly, this is `partial_sum`. The top of the callstack pane contains a list of six symbols:
![Callstack symbols](./figures/callstack_symbols.png)
These symbols allow us to navigate the code as explained in the next section.

* finally, the **SOURCE** pane (highlighted in blue) just below the **BREAKPOINTS** pane contains a copy of the code, again with the currently active line of code highlighted in gray.

### Stepping through code
Obviously, being able to see the values of all variables at a certain point of the code execution is very useful. In addition, the JupyterLab debugger allows us to step through the code in a systematic way. First, observe that we can set more than one breakpoint. For this, click on line 23 to set another breakpoint inside the loop over `k`. Next, click on the 'Play' symbol

![Play symbol](./figures/play_symbol.png)

in the CALLSTACK pane. You should see that execution advances to the next breakpoint, as shown in the following figure:

![Active debugger](./figures/active_debugger_2.png)

As the VARIABLES pane shows, the values of the variables have changed and there is a new variable `k`, which has the value 0 since this is the first iteration of this loop. Clicking on the 'Play' symbol again will advance to the next breakpoint, which is line 21 in the code (if `j=0` then the innermost loop will only contain a single iteration for `k=0`). Do this and observe that the value of `j` has now changed to 1 since we advanced to the next iteration of the outermost loop.

Next, consider the 'Next' Symbol:

![Next symbol](./figures/next_symbol.png)

Instead of advancing to the next breakpoint, this will simply execute the next line of code. Do this a couple of times and observe how the loop variables `j` and `k` change as Python executes one line after the other.

Note that line 23 contains a call to the `square()` function. When stepping through the code with the 'Next' button the debugger will 'step over' the function call. To avoid this, we can use the 'Step in' button

![Step in symbol](./figures/step_in_symbol.png)

Clicking on this button will enter the function, thus allowing us to inspect the inner workings of that function as well.

The result should look like this:

![Stepping inside a function](./figures/debuggerview_stepin.png)

Observe that VARIABLES pane now shows the local variables of the `square()` function. The CALLSTACK pane confirms that Python is currently executing the `square()` function, which is shown at the top of the callstack. We can, however, click on one of the other entries in this list. Doing so will tell us what the state of the program was before entering the `square()` function. Try it out by selecting the `partial_sum()` entry from the list.

Clicking the 'Step out' button

![Step out symbol](./figures/step_out_symbol.png)

will execute the rest of the current function and return to the calling function. In our case, this will leave the `square()` function and continue with the `partial_sum()` function.

Finally, consider the 'Stop' button:

![Stop Symbol](./figures/stop_symbol.png)

Clicking this button will exit the debugger. The calling cell should no longer be marked with a `[*]`.

## So what is wrong with the code?
Stepping through the code line by line and inspecting the values of the variables tells us what is wrong with the code: we observe that when we set `b[j]=0` in line 21, this will also change `a[j]` to zero! In other words, if we change the array `b` this will also modify `a`. The reason for this is that `b=np.asarray(a)` does not create a copy of the array `a` but instead makes `b` a reference to `a`. To fix the problem, we need to change line 11 to

```Python
b = np.array(a)
```

which creates a new copy of `a`. Make this change to the code and step through the corrected code with the debugger, inspecting the variables `a` and `b`. Note that now the code behaves as expected and the entries of `a` do not get overwritten whenever we modify `b`.

## Try it yourself
Use the JupyterLab debugger to step through some code that you have written in the last semester. Try out different methods as discussed above.